In [ ]:
from numpy import *
#from mayavi import mlab
import matplotlib.pyplot as plt
import random

sim_phi0   = 0.0
sim_theta0 = 0.40
sim_nutation   = 4
sim_precession = 40
sim_precangle =  0.20

In [ ]:
sim_theta  = 0.0
sim_phi    = 0.0
sim_angvel = np.array([0.0,0.0,0.0])

sim_pos = np.array([0.0,0.0,0.0])

# Note:
#
# sim_nozzle%theta and sim_nozzle%phi are the 
# jet angles in simulation coordinates
#
# thetajet and phijet are the polar angles of the jet axis in
# precession cone coordinates
#

# base vectors for precession cone
exprime = np.array([ cos(sim_phi0)*cos(sim_theta0), sin(sim_phi0)*cos(sim_theta0), -sin(sim_theta0)])
eyprime = np.array([-sin(sim_phi0)                , cos(sim_phi0)                , 0.])
ezprime = np.array([ cos(sim_phi0)*sin(sim_theta0), sin(sim_phi0)*sin(sim_theta0),  cos(sim_theta0)])

ex = np.array([1.0,0.0,0.0])
ey = np.array([0.0,1.0,0.0])
ez = np.array([0.0,0.0,1.0])
    
# precession axis in simulation coordinates
rcart = np.array([ sin(sim_theta)*cos(sim_phi), sin(sim_theta)*sin(sim_phi), cos(sim_theta)])

# jet axis in precession cone coordinates
r = np.array([sum(rcart[:]*exprime[:]), sum(rcart[:]*eyprime[:]), sum(rcart[:]*ezprime[:])])

# jet angles in precession cone base
thetajet=arccos(r[2]/sqrt(sum(r[:]**2)))

if (r[1]>0 or r[0]>0):
    phijet=2.0*arctan(r[1]/(r[0] + sqrt(r[0]**2 + r[1]**2)))
else:
    phijet=pi


# base vectors in precession cone coordinates
phihat = cross(r[:],ez[:])

if (sum(phihat[:]**2)>0):
    phihat[:]=phihat[:]/sqrt(sum(phihat[:]**2))
else:
    phihat[:]=ey[:]

thetahat=cross(phihat[:],ez[:])

# velocity in precession cone coordinates
v = np.array([sum(exprime[:]*sim_angvel[:]), \
     sum(eyprime[:]*sim_angvel[:]), \
     sum(ezprime[:]*sim_angvel[:])])

sim_vec=ezprime[:]
sim_vec_old=sim_vec[:]


print exprime, eyprime, ezprime


dt = 1.0
plotdt = 1.0
tend = 10000.0
sim_duration = tend
ts = np.linspace(0.0, tend, tend/dt)
dr_dt = ts[1]-ts[0]
vecs, thetajets, phijets = [], [], []
thetas, phis = [], []
ts_plot = []
for t in ts:
    
    phihat=cross(ez[:], r[:])
    if (sqrt(sum(phihat[:]**2))>0) :
        phihat=phihat/sqrt(sum(phihat[:]**2))
    else:
        phihat=eyprime[:]


    thetahat=cross(phihat[:],ez[:])

    # Now express velocity in new coordinate system
    vphi=sum(v[:]*phihat[:])
    vtheta=sum(v[:]*thetahat[:])
    #print vphi, vtheta

    # Random walk step
    sigma=2.0*pi*exp(-(thetajet/sim_precangle)**6)
    rn = random.random()
    psi=(rn - 0.5)*sigma - 0.5*pi

    # Add to velocity
    vphi   =  vphi + sqrt(sim_nutation*sim_precession*dr_dt/sim_duration)*cos(psi)
    vtheta =  vtheta + sqrt(sim_nutation*sim_precession*dr_dt/sim_duration)*sin(psi)

    # And normalize
    dummyn=sqrt(vphi**2 + vtheta**2)
    vphi=vphi/dummyn
    vtheta=vtheta/dummyn

    # Now move jet axis and normalize
    r=r[:] + vphi*phihat[:]*dr_dt*sim_precession/sim_duration + vtheta*thetahat[:]*dr_dt*sim_precession/sim_duration
    r=r[:]/sqrt(sum(r[:]**2))

    # New velocity in coordinates relative to cone axis
    v=vphi*phihat[:] + vtheta*thetahat[:]

    # Finally, calculate new angles
    thetajet=arccos(r[2])
    if (abs(r[1])>0) :
        phijet=2.0*arctan(r[1]/(r[0] + sqrt(r[0]**2 + r[1]**2)))
    else:
        phijet=pi

    # Now express in simulation coordinates
    jprime=np.array([cos(phijet)*sin(thetajet), sin(phijet)*sin(thetajet),cos(thetajet)])
    jetaxis=exprime[:]*jprime[0] + eyprime[:]*jprime[1] + ezprime[:]*jprime[2]

    # sim_theta and sim_phi are measured in simulations coordinates
    sim_theta=arccos(jetaxis[2]/sqrt(sum(jetaxis[:]**2)))
    if (jetaxis[1]>-sqrt(jetaxis[0]**2 + jetaxis[1]**2)) :
        sim_phi=2.0*arctan(jetaxis[1]/(jetaxis[0] + sqrt(jetaxis[0]**2 + jetaxis[1]**2)))
    else:
        sim_phi=pi


    # angular velocity for precession
    sim_angvel=cross(jetaxis[:],sim_vec[:])/dr_dt

    # move this to jiggle
    #sim_vec_old=sim_vec[:]
    sim_vec=jetaxis[:]
    #sim_pos_old=sim_pos[:]
    #sim_pos=sim_pos[:]+dr_dt*1.e8*np.array([0.3,0.,0.6])
    #print sim_vec
    #if True:
    if np.mod(t, plotdt) <= dr_dt:
        thetajets.append(thetajet)
        phijets.append(phijet)
        thetas.append(sim_theta)
        phis.append(sim_phi)
        vecs.append(sim_vec)
        ts_plot.append(t)
        #print '%10.3e, %6.3f, %6.3f, %6.3f' % (sigma, cos(psi), sin(psi), thetajet)
thetajets = np.array(thetajets)
phijets = np.array(phijets)

In [ ]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 12, 8

fig = plt.figure(figsize=(15,15))
ax = plt.subplot(111, polar=True)
ax.scatter(sim_phi0, sim_theta0, marker='o', color='r', s=20)
ax.scatter(phis[0], thetas[0], marker='x', color='r')
for ind in range(len(phis)-1):
    ax.plot(phis[ind:ind+2], thetas[ind:ind+2], color='r', linewidth=0.5, alpha = (float(ind)/len(phis)+0.1)/1.1)
#ax.plot(phis, thetas, color='r', linewidth=0.0, alpha=0.5, marker='.')
ax.scatter(phijets, thetajets, color='b', alpha=0.5, s=1, marker='.')

preccircle = np.linspace(0,2*np.pi, 100)
ax.plot(preccircle, [sim_precangle]*100, color='b', linewidth=1)
#ax.set_rmax(0.2*np.pi)
ax.grid(True)


In [ ]:
n, bins, patches = plt.hist(thetajets, 40, alpha=0.5)
bin_centers = (bins[1:]+bins[:-1])/2.0
print bin_centers.shape, thetajets.shape
plt.plot(bin_centers, n/bin_centers/15)
#plt.xlim(0.0, 0.15)
#plt.ylim(0, 1000)
#n, bins, patches = plt.hist(thetajets, 50, range=(0.02, 0.15))
#print bins,n

In [ ]:
ax1 = plt.subplot(211)
ax1.plot(ts_plot, thetajets)
ax1.set_ylabel('theta')
ax2 = plt.subplot(212)
ax2.plot(ts_plot, phijets)
ax2.set_ylim(-np.pi, np.pi)
ax2.set_ylabel('phi')


In [ ]:
# render the sphere mesh
us, vs = np.mgrid[0:2*np.pi:60j, 0:np.pi:30j]
x=np.cos(us)*np.sin(vs)
y=np.sin(us)*np.sin(vs)
z=np.cos(vs)

# White background and turn on antialiasing
fig = mlab.figure(bgcolor=(1,1,1))
fig.scene.render_window.aa_frames = 8

# Plot the sphere's mesh
for a,b,c in zip(x.T, y.T, z.T) + zip(x, y, z):
    mlab.plot3d(a, b, c, color=(1,0,0), tube_radius=0.003, opacity=0.05)

In [ ]:
vecs = np.array(vecs)

#vecs = np.array([[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
u, v, w = vecs.T
zero = np.zeros_like(u)

colors = [(t/tend, t/tend, t/tend) for t in ts_plot]
head_size= 0.1
h, b = head_size, 1.0-head_size

for ind, (i, j, k) in enumerate(zip(u, v, w)):
    mlab.quiver3d(b*i, b*j, b*k, h*i, h*j, h*k, color=colors[ind],
              mode='cone', scale_factor=0.3, resolution=20)
    #mlab.plot3d([0, b*i], [0, b*j], [0, b*k], color=colors[ind], tube_radius=0.005, opacity=0.6)

mlab.show()

In [ ]:
print s